In [3]:
import os
import pymongo
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

MONGO_DB_USER = os.environ["MONGO_DB_USER"]
MONGO_DB_PASSWORD = os.environ["MONGO_DB_PASSWORD"]
MONGO_DB_HOST = os.environ["MONGO_DB_HOST"]
MONGO_DB_PORT = os.environ["MONGO_DB_PORT"]

client = pymongo.MongoClient(
    f"mongodb://{MONGO_DB_USER}:{MONGO_DB_PASSWORD}@{MONGO_DB_HOST}:{MONGO_DB_PORT}/"
)

silver_db = client.silver
silver_events = silver_db.events
silver_games = silver_db.games

In [351]:
print("Most recent event in the Database:")

most_recent_event = silver_events.find_one(sort=[("Date", -1)])

print(most_recent_event["Date"])

Most recent event in the Database:
2024-07-09 00:00:00


## Biggest Winners

We'll be analysing the Double Tournament era here, which stated in 01.02.2024 according to Chess.com.

Source: https://www.chess.com/article/view/titled-tuesday

In [354]:
match_double_elimination_era = {
    "$match": {
        "Date": {"$gte": datetime(2022, 1, 31)},
    }
}

winners = silver_events.aggregate(
    [
        match_double_elimination_era,
        {
            "$project": {
                "results": {
                    "$filter": {
                        "input": "$results",
                        "as": "r",
                        "cond": {"$eq": ["$$r.position", 1]},
                    }
                }
            }
        },
    ]
)

dict_winners = {}

for i in winners:
    for winner in i["results"]:
        dict_winners.setdefault(winner["player"], 0)
        dict_winners[winner["player"]] += 1

print(f"Number of Events: {sum(dict_winners.values())}\n")

print(f"Players with most wins")
for key, value in sorted(dict_winners.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"{key:>22}: {value}")

Number of Events: 244

Players with most wins
      Nakamura, Hikaru: 55
     Andreikin, Dmitry: 16
       Carlsen, Magnus: 16
         Dubov, Daniil: 11
     Firouzja, Alireza: 9
        Sarana, Alexey: 8
   Duda, Jan-Krzysztof: 8
    Fedoseev, Vladimir: 6
     Bortnyk, Olexandr: 6
        Xiong, Jeffery: 5


During the first data exploration of this dataset it has been found that some players have multiple accounts in the database, this has been corrected during the silver ETL phase and left here for reference.

```python
dict_multiple_accounts = {
    "Nakamura, Hikaru": "Hikaru",
    "Carlsen, Magnus": "MagnusCarlsen",
    "Firouzja, Alireza": "Firouzja2003",
    "Sarana, Alexey": "mishanick",
    "Andreikin, Dmitry": "FairChess_on_YouTube",
    "Duda, Jan-Krzysztof": "Polish_fighter3000",
}

for key, value in dict_multiple_accounts.items():
    dict_winners[key] += dict_winners[value]
    del dict_winners[value]

print(f"Players with most wins")
for key, value in sorted(dict_winners.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"{key:>22}: {value}")
```

In [395]:
# Searching if there's any other accounts that may belong to the top players

players_to_search = ["magnus", "carlsen", "hikaru", "nakamura"]

for player in players_to_search:
    total_games_per_player = silver_games.aggregate(
        [
            match_double_elimination_era,
            {"$project": {"White": 1, "Black": 1, "_id": 0}},
            {"$project": {"values": {"$objectToArray": "$$ROOT"}}},
            {"$unwind": "$values"},
            {"$group": {"_id": "$values.v", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}},
            {"$match": {"_id": {"$regex": player, "$options": "i"}}},
        ]
    )

    print(f"Player: {player}")
    for i in total_games_per_player:
        print(f"Games: {i['count']} - Player: {i['_id']}")
    print("\n")

Player: magnus
Games: 758 - Player: Carlsen, Magnus
Games: 5 - Player: CarlMagnuson
Games: 2 - Player: Magnuson, Carl F


Player: carlsen
Games: 758 - Player: Carlsen, Magnus
Games: 31 - Player: carlsenCamacho
Games: 11 - Player: carlsen1001
Games: 7 - Player: BetonCarlsen


Player: hikaru
Games: 1999 - Player: Nakamura, Hikaru
Games: 18 - Player: hikaruchija


Player: nakamura
Games: 1999 - Player: Nakamura, Hikaru




### Notes

No other accounts were found that belong to the top players.

## Biggest upsets

An upset is when one player's rating is lower than the other player's.

Here we'll be looking into which were the biggest upsets in the Double-Tournament Era.

In [355]:
upsets = (
    silver_games.find(
        {
            "WhiteElo": {"$gt": 1000},
            "BlackElo": {"$gt": 1000},
            "Date": {"$gte": datetime(2022, 1, 1)},
        }
    )
    .limit(10)
    .sort({"Upset": 1})
)

upsets = pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

upsets.loc[:, "Chances of Upset %"] = 1 / (1 + 10 ** (-upsets["Upset"] / 400))

upsets.style.format({"Chances of Upset %": "{:.4%}", "Upset": "{:.0f}"})

,Event,Upset,White,WhiteElo,Black,BlackElo,Chances of Upset %
0,Titled Tuesday Blitz December 20 Early 2022,-1619,Susal_De_Silva,2808,"Kazgildy, Fariza",1189,0.0090%
1,Titled Tuesday Blitz October 18 Late 2022,-1546,"Eisa Mohmood, Alkhoori",1245,"Bordi, Kevin",2791,0.0136%
2,Titled Tuesday Blitz December 20 Early 2022,-1536,Kopylov_Alex,2790,"Mone, Ved",1254,0.0145%
3,Titled Tuesday Blitz December 20 Early 2022,-1463,The_Machine04,2652,"Kazgildy, Fariza",1189,0.0220%
4,Titled Tuesday Blitz April 26 Early 2022,-1414,"Bashirli, Saadat",1173,"Bachmann, Axel",2587,0.0292%
5,Titled Tuesday Blitz December 20 Early 2022,-1393,"Benamar, Benious",1136,lemayvega,2529,0.0329%
6,Titled Tuesday Blitz December 20 Early 2022,-1350,CDOEspiao,2486,"Benamar, Benious",1136,0.0422%
7,Titled Tuesday Blitz December 20 Early 2022,-1328,"Castro Andrade, Seirbert Cristian",1364,"Xiong, Jeffery",2692,0.0478%
8,Titled Tuesday Blitz December 20 Early 2022,-1311,"Yildiz, Ayse Emek",1702,OparinGrigoriy,3013,0.0528%
9,Titled Tuesday Blitz January 03 Late 2023,-1309,"Gaehwiler, Gabriel",2404,"Eisa Mohmood, Alkhoori",1095,0.0534%


### Notes

In the tables above we can see that the biggest upset in this tournament had an elo difference of 1619, this had a chance of happening of just 0.009% ([Source](https://fivethirtyeight.com/features/introducing-nfl-elo-ratings/)).

### Carlsen and Nakamura Upsets

In [356]:
print(
    f"Number of games player by Carlsen: {silver_games.count_documents({'$or': [{'White': 'Carlsen, Magnus'}, {'Black': 'Carlsen, Magnus'}]})}"
)

upsets = (
    silver_games.find(
        {
            "$or": [
                {"White": {"$in": ["Carlsen, Magnus"]}},
                {"Black": {"$in": ["Carlsen, Magnus"]}},
            ],
            "WhiteElo": {"$gt": 1000},
            "BlackElo": {"$gt": 1000},
        }
    )
    .limit(5)
    .sort({"Upset": 1})
)

carlsen_upsets = pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

carlsen_upsets.loc[:, "Chances of Upset %"] = 1 / (1 + 10 ** (-carlsen_upsets["Upset"] / 400))

carlsen_upsets.style.format({"Chances of Upset %": "{:.4%}", "Upset": "{:.0f}"})

Number of games player by Carlsen: 787


,Event,Upset,White,WhiteElo,Black,BlackElo,Chances of Upset %
0,Late-Titled-Tuesday-Blitz-June-25-2024,-616,"Carlsen, Magnus",3164,WPWPWP-2015,2548,2.8032%
1,Late-Titled-Tuesday-Blitz-July-02-2024,-612,subham777,2545,"Carlsen, Magnus",3157,2.8666%
2,Titled Tuesday Blitz March 21 Early 2023,-475,"Carlsen, Magnus",2852,"Tillyaev, Ulugbek",2377,6.0978%
3,Late-Titled-Tuesday-Blitz-February-20-2024,-465,ProphetReborn,2860,"Carlsen, Magnus",3325,6.4359%
4,Titled Tuesday Blitz January 31 Late 2023,-440,"Erdogdu, Mert",2413,"Carlsen, Magnus",2853,7.3588%


In [358]:
print(
    f"Number of games player by Hikaru Nakamura: {silver_games.count_documents({'$or': [{'White': 'Nakamura, Hikaru'}, {'Black': 'Nakamura, Hikaru'}]})}"
)

upsets = (
    silver_games.find(
        {
            "$or": [
                {"White": {"$in": ["Nakamura, Hikaru"]}},
                {"Black": {"$in": ["Nakamura, Hikaru"]}},
            ],
            "WhiteElo": {"$gt": 1000},
            "BlackElo": {"$gt": 1000},
        }
    )
    .limit(5)
    .sort({"Upset": 1})
)

nakamura_upsets = pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

nakamura_upsets.loc[:, "Chances of Upset %"] = 1 / (1 + 10 ** (-nakamura_upsets["Upset"] / 400))

nakamura_upsets.style.format({"Chances of Upset %": "{:.4%}", "Upset": "{:.0f}"})

Number of games player by Hikaru Nakamura: 3127


,Event,Upset,White,WhiteElo,Black,BlackElo,Chances of Upset %
0,Titled Tuesday Blitz August 15 Early 2023,-1180,"Nakamura, Hikaru",3255,"Nugumanov, Bakhtiyar",2075,0.1121%
1,Titled Tuesday Blitz May 10 Early 2022,-1054,"Nakamura, Hikaru",2850,"Aslanov, Umid",1796,0.2312%
2,*** Titled Tuesday Blitz AM,-964,"Nakamura, Hikaru",3134,politeplayer,2170,0.3875%
3,***Titled Tuesday 3|2 Blitz,-793,"Nakamura, Hikaru",2890,proxy6,2097,1.0304%
4,Titled Tuesday Blitz August 15 Early 2023,-761,"Benedetti, Julio",2510,"Nakamura, Hikaru",3271,1.2362%


### Notes on Nakamura and Carlsen Upsets

Magnus Carlsen never had a huge upset, having lost to someone with a rating difference of 475 and a chance of upset of 6%.

Hikaru Nakamura had more upsets, losing many times to players with over 500 difference in rating and a couple of times with chances of upsets below 1%.

## Who played the most games

In [359]:
total_games = silver_games.aggregate([match_double_elimination_era, {"$count": "total_games"}])

print(
    "Number of games played in the 2022 Double Elimination era:", total_games.next()["total_games"]
)

Number of games played in the 2022 Double Elimination era: 430825


In [396]:
total_games_per_player = silver_games.aggregate(
    [
        match_double_elimination_era,
        {"$project": {"White": 1, "Black": 1, "_id": 0}},
        {"$project": {"values": {"$objectToArray": "$$ROOT"}}},
        {"$unwind": "$values"},
        {"$group": {"_id": "$values.v", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10},
    ]
)

print("Players who played the most games in the 2022 Double Elimination era:")
for i in total_games_per_player:
    print(f"Games: {i['count']} - Player: {i['_id']}")

Players who played the most games in the 2022 Double Elimination era:
Games: 2236 - Player: Andreikin, Dmitry
Games: 2032 - Player: Sarana, Alexey
Games: 1999 - Player: Nakamura, Hikaru
Games: 1686 - Player: Zhigalko, Sergei
Games: 1655 - Player: Kamsky, Gata
Games: 1580 - Player: Bortnyk, Olexandr
Games: 1503 - Player: Martinez Alcantara, Jose Eduardo
Games: 1456 - Player: Rustemov, Alexander
Games: 1445 - Player: Paravyan, David
Games: 1438 - Player: Vlassov, Nikolai


## Who played the most editions

In [517]:
total_games_per_player = silver_games.aggregate(
    [
        {
            "$project": {
                "Event": 1,
                "White": {"Event": "$Event", "player": "$White"},
                "Black": {"Event": "$Event", "player": "$Black"},
            }
        },
        {"$project": {"Event": 1, "players": {"$setUnion": [["$White"], ["$Black"]]}}},
        {"$unwind": "$players"},
        {"$replaceRoot": {"newRoot": "$players"}},
        {
            "$group": {
                "_id": {"event": "$Event", "player": "$player"},
            }
        },
        {"$replaceRoot": {"newRoot": "$_id"}},
        {"$group": {"_id": "$player", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10},
    ]
)

print("Players who played the most Titled Tuesday Editions in the 2022 Double Elimination era:")
for i in total_games_per_player:
    print(f"Editions: {i['count']} - Player: {i['_id']}")

Players who played the most Titled Tuesday Editions in the 2022 Double Elimination era:
Editions: 248 - Player: Andreikin, Dmitry
Editions: 229 - Player: Nakamura, Hikaru
Editions: 219 - Player: Sarana, Alexey
Editions: 189 - Player: Zhigalko, Sergei
Editions: 186 - Player: Kamsky, Gata
Editions: 180 - Player: Janaszak, Dawid
Editions: 179 - Player: Bortnyk, Olexandr
Editions: 177 - Player: Rustemov, Alexander
Editions: 174 - Player: Vlassov, Nikolai
Editions: 170 - Player: Martinez Alcantara, Jose Eduardo
